In [6]:
import os
import json
import base64
import cv2  
from ultralytics import YOLO
import shutil

model = YOLO('yolo11n.pt')

root = os.getcwd()+'/Traffic Dataset'

clss = ['Car', 'Number Plate', 'Number Plate', 'Two Wheeler', 'Auto', 'Bus', 'Truck']

subFolders = ['train','val']

for subFolder in subFolders:
    imgDir = os.fsencode(f"{root}/images/{subFolder}")

    for file in os.listdir(imgDir):
        filename = os.fsdecode(file)
        img = cv2.imread(f"{root}/images/{subFolder}/{filename}")
        ih, iw, c = img.shape
        labelFName = '.'.join(filename.split('.')[:-1]) + '.txt'
        outName = '.'.join(filename.split('.')[:-1]) + '.json'
        with open(f"{root}/labels/{subFolder}/{labelFName}",'r') as fh:
            lblDataLines = fh.readlines()
        lblme = {}
        lblme['version'] = '5.5.0'
        lblme['flags'] = {}
        lblme['shapes'] = []
        for lblData in lblDataLines:
            [id, nx, ny, nw, nh] = [float(v) for v in lblData.split(' ')]
            x1 = nx*iw - nw*iw/2
            y1 = ny*ih - nh*ih/2
            x2 = nx*iw + nw*iw/2
            y2 = ny*ih + nh*ih/2
            shape = {}
            shape['label'] = clss[int(id)]
            shape['group_id'] = None
            shape['description'] = ''
            shape['shape_type'] = 'rectangle'
            shape['flags'] = {}
            shape['mask'] = None
            shape['points'] = [[x1,y1],[x2,y2]]
            lblme['shapes'].append(shape)
        results = model.predict(f"{root}/images/{subFolder}/{filename}")
        for r in results:
            persons = [box for box in r.boxes if int(box.cls.item()) in [0]]
            for m in persons:
                [x1,y1,x2,y2] = m.xyxy.tolist()[0]
                shape = {}
                shape['label'] = 'Person'
                shape['group_id'] = None
                shape['description'] = ''
                shape['shape_type'] = 'rectangle'
                shape['flags'] = {}
                shape['mask'] = None
                shape['points'] = [[x1,y1],[x2,y2]]
                lblme['shapes'].append(shape)
        lblme['imagePath'] = filename
        with open(f"{root}/images/{subFolder}/{filename}",'rb') as fh:
            lblme['imageData'] = str(base64.b64encode(fh.read()))[1:-1]
        lblme['imageHeight'] = ih
        lblme['imageWidth'] = iw
        with open(f"{root}/res/{outName}",'w') as fh:
            json.dump(lblme, fh)
        shutil.copyfile(f"{root}/images/{subFolder}/{filename}",f"{root}/res/{filename}")


image 1/1 C:\Users\Usuario\otsedom.github.io\TRAINING2\Traffic Dataset\images\train\00 (10).jpg: 480x640 1 person, 2 buss, 1 truck, 42.5ms
Speed: 1.0ms preprocess, 42.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\Usuario\otsedom.github.io\TRAINING2\Traffic Dataset\images\train\00 (100).jpg: 384x640 1 person, 6 cars, 2 trucks, 3 traffic lights, 28.7ms
Speed: 1.0ms preprocess, 28.7ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\Usuario\otsedom.github.io\TRAINING2\Traffic Dataset\images\train\00 (102).jpg: 384x640 1 person, 15 cars, 1 truck, 2 traffic lights, 1 handbag, 29.5ms
Speed: 2.0ms preprocess, 29.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 C:\Users\Usuario\otsedom.github.io\TRAINING2\Traffic Dataset\images\train\00 (103).jpg: 384x640 13 cars, 1 truck, 2 traffic lights, 26.7ms
Speed: 1.0ms preprocess, 26.7ms inference, 0.0ms postprocess per image at shape (1, 3, 3